In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import numpy as np
import pymongo 

import datetime, time
from datetime import datetime

import os

In [2]:
client = pymongo.MongoClient()

db = client['fleet']

fleet1 = db['TestFleet']

spark = SparkSession.builder.appName("App").getOrCreate()

In [3]:
def cleanDF(df):
    return df.select("Date", "Latitude", "Longitude", "Speed").dropDuplicates().sort("Date").withColumnRenamed("Speed", "Velocity")
    #return df.drop("Events", "Vision Events", "Geofences", "Location", "Odometer", "Linear G", "Lateral G", "Road Speed").dropDuplicates().sort("Date").withColumnRenamed("Speed", "velocity")

In [4]:
data_dir = 'D:\\Telematics\\Car Track\\Data\\Joined'
os.listdir(data_dir)


for file in os.listdir(data_dir):
    fp = os.path.join(data_dir, file)
    df = spark.read.csv(fp, inferSchema=True, header=True)
    df_clean = cleanDF(df)

D:\Telematics\Car Track\Data\Joined\KF44LNGP.csv
+-------------------+----------+---------+--------+
|               Date|  Latitude|Longitude|velocity|
+-------------------+----------+---------+--------+
|2022-07-01 05:40:02|-25.995033|28.176716|       0|
|2022-07-01 06:16:53|-25.995033|28.176716|       0|
|2022-07-01 08:02:58|-25.995033|28.176716|       0|
|2022-07-01 08:03:11|  -25.9951|28.176683|       0|
|2022-07-01 08:03:40|-25.995083|28.176735|       0|
|2022-07-01 08:04:10|-25.995083|28.176735|       0|
|2022-07-01 08:04:40|-25.995083|28.176735|       0|
|2022-07-01 08:05:10|-25.995083|28.176735|       0|
|2022-07-01 08:05:40|-25.995083|28.176735|       0|
|2022-07-01 08:06:10|-25.995083|28.176735|       0|
|2022-07-01 08:06:40|-25.995083|28.176735|       0|
|2022-07-01 08:07:10|-25.995083|28.176735|       0|
|2022-07-01 08:07:40|-25.995083|28.176735|       0|
|2022-07-01 08:08:10|-25.995083|28.176735|       0|
|2022-07-01 08:08:40|-25.995083|28.176735|       0|
|2022-07-01 08:

In [4]:
data_dir = 'D:\\Telematics\\Car Track\\Data\\Joined'
file = os.path.join(data_dir, os.listdir(data_dir)[0])
df = spark.read.csv(file, inferSchema=True, header=True)
df_clean = cleanDF(df)

group = df_clean.withColumn("Minute", minute(df_clean.Date)).groupBy('Minute', window("Date", "10 seconds")).agg(
    mean("velocity").alias("velocity"),
    mean("Longitude").alias("Longitude"),
    mean("Latitude").alias("Latitude"),
    expr("collect_list(velocity)").alias("velocity list")
)

df_resampled = group.withColumn("Start", group.window.start).withColumn("End", group.window.end).sort("Start")
df_resampled = df_resampled.withColumn("Date",from_unixtime((unix_timestamp(df_resampled.Start) + unix_timestamp(df_resampled.End))/2, "yyyy-MM-dd HH:mm:ss"))
v_clean = df_resampled.select(["Date", "Start", "End", "velocity", "latitude", "longitude"]).withColumn("Date", to_timestamp(df_resampled.Date, "yyyy-MM-dd HH:mm:ss"))

In [5]:
v_clean.select('latitude').collect()

[Row(latitude=-25.995033),
 Row(latitude=-25.995033),
 Row(latitude=-25.995033),
 Row(latitude=-25.9951),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995083),
 Row(latitude=-25.995068),
 Row(latitude=-25.995068),
 Row(latitude=-25.995001),
 Row(latitude=-25.995001),
 Row(latitude=-25.995001),
 Row(latitude=-25.995001),
 Row(latitude=-25.995001),
 Row(latitude=-25.995001),
 Row(latitude=-25.995001),
 Row(latitude=-25.995001),
 Row(latitude=-25.995001),
 Row(latitude=-25.995001),
 Row(latitude=-25.9949),
 Row(latitude=-25.9947825),
 Row

In [6]:
def getISODate(df):
    return np.array([x.Date.isoformat() + "Z" for x in df.select("Date").collect()])

In [3]:
def getDate(df):
    return [x.Date for x in df.select("Date").collect()]

In [4]:
def getParameter(df, param):
    match param:
        case "Latitude":
            return np.array([x.Latitude for x in df.select("Latitude").collect()])
        case "Longitude":
            return np.array([x.Longitude for x in df.select("Longitude").collect()])
        case "velocity":
            return np.array([x.velocity for x in df.select("velocity").collect()])
        case _:
            return None

In [8]:
lat = getParameter(df_clean, "Latitude")
lng = getParameter(df_clean, "Longitude")
velocity = getParameter(df_clean, "velocity")

In [9]:
date = getDate(df_clean)

In [10]:
class tag:
    def __init__(self, numPlate):
        self.numPlate = numPlate
    def getLatTag(self):
        return self.numPlate + "_lat"
    def getVelocityTag(self):
        return self.numPlate + "_V"
    def getLngTag(self):
        return self.numPlate + "_lng"
    def getNumPlate(self):
        return self.numPlate

In [14]:
car = tag("KN34FKGP")
car.getVelocityTag()

'KN34FKGP_V'

In [40]:
tag_lat = "KK65XRGP_lat"
tag_lng = "KK65XRGP_lng"
tag_vel = "KK65XRGP_V"

In [16]:
def latCollection(date, lat, tag):
    col = []
    for i in range(len(date)):
        temp = {
            "tag": tag, 
            #"date": date[i],
            "timestamp": date[i],
            "latitude": lat[i]
        }
        
        col.append(temp)
    return col

In [17]:
def lngCollection(date, lng, tag):
    col = []
    for i in range(len(date)):
        temp = {
            "tag": tag, 
            #"date": date[i],
            "timestamp": date[i],
            "longitude": lng[i]
        }
        
        col.append(temp)
    return col

In [18]:
def velCollection(date, velocity, tag):
    col = []
    for i in range(len(date)):
        temp = {
            "tag": tag, 
            #"date": date[i],
            "timestamp": date[i],           
            "velocity": int(velocity[i])
        }
        
        col.append(temp)
    return col

In [19]:
insertLat = latCollection(date, lat, car.getLatTag())
insertLng = lngCollection(date, lng, car.getLngTag())
insertVel = velCollection(date, velocity, car.getVelocityTag())

In [14]:
fleet1.insert_one({
    "tag":"YBC193GP_V",
    "timestamp": datetime.now(),
    "velocity": 45
})

In [ ]:
temp = insertLat[0]
temp

In [20]:
fleet1.insert_many(insertLat)

In [ ]:
for x in fleet1.find():
    print(x)

In [21]:
fleet1.insert_many(insertLng)

In [22]:
fleet1.insert_many(insertVel)

In [23]:
for x in fleet1.find({"tag": "KK65XRGP_V"}):
    print(x)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'timestamp': datetime.datetime(2022, 7, 20, 10, 48, 24), 'tag': 'KK65XRGP_V', '_id': ObjectId('62fe7ae1f29ed8c4ec2ff730'), 'velocity': 43}
{'timestamp': datetime.datetime(2022, 7, 20, 10, 48, 37), 'tag': 'KK65XRGP_V', '_id': ObjectId('62fe7ae1f29ed8c4ec2ff731'), 'velocity': 53}
{'timestamp': datetime.datetime(2022, 7, 20, 10, 48, 47), 'tag': 'KK65XRGP_V', '_id': ObjectId('62fe7ae1f29ed8c4ec2ff732'), 'velocity': 77}
{'timestamp': datetime.datetime(2022, 7, 20, 10, 48, 56), 'tag': 'KK65XRGP_V', '_id': ObjectId('62fe7ae1f29ed8c4ec2ff733'), 'velocity': 92}
{'timestamp': datetime.datetime(2022, 7, 20, 10, 49, 8), 'tag': 'KK65XRGP_V', '_id': ObjectId('62fe7ae1f29ed8c4ec2ff734'), 'velocity': 81}
{'timestamp': datetime.datetime(2022, 7, 20, 10, 49, 18), 'tag': 'KK65XRGP_V', '_id': ObjectId('62fe7ae1f29ed8c4ec2ff735'), 'velocity': 95}
{'timestamp': datetime.datetime(2022, 7, 20, 10, 49, 38), 'tag': 'KK65XRGP_V', '_id': ObjectId('62fe7ae1f29ed8c4ec2ff736'), 'velocity': 100}
{'timestamp': dateti

In [19]:
print(len([x for x in fleet1.find()]))

87386


In [45]:
df_clean.count()*len(df_clean.select("Latitude","Longitude", "velocity").columns)

86976

In [23]:
fleet1.find({"tag": "KK65XRGP_lat"}).

AttributeError: 'Cursor' object has no attribute 'pretty'

In [25]:
fleet1.explain("executionStats")

TypeError: 'Collection' object is not callable. If you meant to call the 'explain' method on a 'Collection' object it is failing because no such method exists.